# **EVA** TEXT CLASSIFIER

![img_ppal](https://www.innovacion-tecnologia.com/wp-content/uploads/2020/09/Historia-de-los-humanoides.jpg)

## *ÍNDICE* 
---
0. INTRODUCCIÓN
1. MACHINE LEARNING
    - 1.1. Librerias, constantes y carga de datos
    - 1.2. Creación y comparación de modelos
    - 1.3. Resultados/predicciones
    - 1.4. Observaciones
2. **DEEP LEARNING**
    - 2.1. Librerias, constantes y carga de datos
    - 2.2. Creación y comparación de modelos
    - 2.3. Resultados/predicciones
    - 2.4. Observaciones
---

# ***DEEP LEARNING: NLP transformers models training***

---

### Librerías

In [ ]:
colab = True
first_time = True

In [ ]:
if colab:
  from google.colab import drive
  drive.mount('/content/drive')
  from drive.MyDrive.ColabNotebooks.HACKATON.hackaton_nuwe_oracle.reto_2.NuweHackatonOracle_NLP_Clf_reto2.src.utils.libreries import *
  from drive.MyDrive.ColabNotebooks.HACKATON.hackaton_nuwe_oracle.reto_2.NuweHackatonOracle_NLP_Clf_reto2.src.utils.functions import *
else: 
  from utils.libreries import *
  from utils.functions import * 

### Definir constantes

In [ ]:
if colab: 
  REL_PATH = "/content/drive/MyDrive/ColabNotebooks/HACKATON/hackaton_nuwe_oracle/reto_2/NuweHackatonOracle_NLP_Clf_reto2/src"
  TRAIN_PATH = REL_PATH + '/data/raw/train.csv'
  TEST_PATH = REL_PATH + '/data/raw/test.csv'
  TRAIN_PROCESSED_PATH = REL_PATH + '/data/processed/train_processed.csv'
else:
  REL_PATH = os.getcwd()
  TRAIN_PATH = REL_PATH + '/data/raw/train.csv'
  TEST_PATH = REL_PATH + '/data/raw/test.csv'
  TRAIN_PROCESSED_PATH = REL_PATH + '/data/processed/train_processed.csv'

In [ ]:
if colab and first_time: 
   nltk.download('wordnet')
   nltk.download('stopwords')
   nltk.download('omw-1.4')
   nltk.download('punkt')
   !pip install emoji==0.6.0
   !pip install vaderSentiment
   !pip install transformers

### Datasets

In [ ]:
df_train = pd.read_csv(TRAIN_PATH)
df_train.head()

,train_idx,text,label,label_text
0,0,i really do recommend this to anyone in need o...,1,positive
1,1,very good every day camera fits nicely in the ...,1,positive
2,2,"but , dollar for dollar , this dvd player is p...",1,positive
3,3,i got this phone yesterday and didn ' t find a...,1,positive
4,4,1 ) price gb of storage,1,positive


In [ ]:
ds_test = pd.read_csv(TEST_PATH)
ds_test.head()

,test_idx,text
0,0,fm receiver it has none
1,1,"the picture quality surprised me , when i firs..."
2,2,great video clip quality for a digital camera ...
3,3,creative did well on its rechargeable battery ...
4,4,i highly recommend this camera to anyone looki...


### Modelo utilizando TRANSFORMERS:


#### Primer modelo usando transformers

In [ ]:
MODEL = f"cardiffnlp/twitter-roberta-base-sentiment"
tokenizer = AutoTokenizer.from_pretrained(MODEL)

In [ ]:
model = TFAutoModelForSequenceClassification.from_pretrained(MODEL)
model.save_pretrained(REL_PATH + '/model/'+ MODEL)

text = "Good night 😊"
encoded_input = tokenizer(text, return_tensors='tf')
output = model(encoded_input)
scores = output[0][0].numpy()
scores = softmax(scores)

All model checkpoint layers were used when initializing TFRobertaForSequenceClassification.

All the layers of TFRobertaForSequenceClassification were initialized from the model checkpoint at cardiffnlp/twitter-roberta-base-sentiment.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFRobertaForSequenceClassification for predictions without further training.


In [ ]:
scores

array([0.00760985, 0.14581189, 0.8465783 ], dtype=float32)

In [ ]:
np.argmax(scores)

2

scores: array (negativo,neutral y positivo)

#### Evaluación del transformer con el train.csv

In [ ]:
textos = df_train['text']
y_pred = []

for texto in textos:
  encoded_input = tokenizer(texto, return_tensors='tf')
  output = model(encoded_input)
  scores = output[0][0].numpy()
  scores = softmax(scores)
  res = np.argmax(scores)
  
  if res==2:
    res=1
  
  y_pred.append(res)

In [ ]:
y_test = list(df_train['label'])

# Evaluar transformer
metricas = eval_metrics(y_pred,y_test)

In [ ]:
file_name = 'transformes_metrics.csv'
dir_file = REL_PATH + '/model/model_metrics'
model_str = str(model)[0:str(model).find('(')]

# Guardar las métricas obtenidas
dict4save(metricas, file_name, dir_file, addcols=True, cols='model', vals=model_str,sep=';')

<class 'list'>
Saved


#### Comparación de los distintos transformers

In [ ]:
metrics,y_pred = text_transformers_eval(texts = df_train['text'],transformer = f"finiteautomata/bertweet-base-sentiment-analysis",folder = 'model', file_name = 'transformes_metrics.csv', dir_file = REL_PATH + '/model/model_metrics')

#### Entrenamiento de transformers con tensorflow

In [ ]:
from transformers import TFRobertaForSequenceClassification, RobertaTokenizer

# Copiar datasets 
train_data = df_train.copy()
train_data = train_data[['text','label']]
test_df = ds_test.copy()
test_df = test_df[['text']]

# Dividir los datos en entrenamiento y validación
train_df, val_df = train_test_split(train_data, test_size=0.2, random_state=42)


# Preprocesar los datos
tokenizer = RobertaTokenizer.from_pretrained(f'roberta-base',num_labels = 2)

# Tokenizar y codificar los conjuntos de datos de entrenamiento y prueba
train_encodings = tokenizer(list(train_df['text']), truncation=True, padding=True)
val_encodings = tokenizer(list(val_df['text']), truncation=True, padding=True)
test_encodings = tokenizer(list(test_df['text']), truncation=True, padding=True)

# Crear conjuntos de datos a partir de las encodings y los labels
train_dataset = tf.data.Dataset.from_tensor_slices((
    dict(train_encodings),
    train_df['label']
))
val_dataset = tf.data.Dataset.from_tensor_slices((
    dict(val_encodings),
    val_df['label']
))
test_dataset = tf.data.Dataset.from_tensor_slices(dict(test_encodings))


In [ ]:
# Configurar el modelo
model = TFRobertaForSequenceClassification.from_pretrained(f'roberta-base', num_labels=2)

optimizer = tf.keras.optimizers.Adam(learning_rate=3e-5)
loss = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)
metric = tf.keras.metrics.SparseCategoricalAccuracy('accuracy')

model.compile(optimizer=optimizer, loss=loss, metrics=[metric])

# Entrenar el modelo
history = model.fit(train_dataset.batch(32), epochs=2, validation_data=val_dataset.batch(32))


In [ ]:
history

### Resultados:

#### Predicciones primera prueba transformers

In [ ]:
textos = ds_test['text']
y_pred = []

for texto in textos:
  encoded_input = tokenizer(texto, return_tensors='tf')
  output = model(encoded_input)
  scores = output[0][0].numpy()
  scores = softmax(scores)
  res = np.argmax(scores)
  
  if res==2:
    res=1
  
  y_pred.append(res)

In [ ]:
df_test = ds_test.copy()

# Nueva columna con la predicción
df_test['prediction'] = y_pred 

df_test.head()

,test_idx,text,prediction
0,0,fm receiver it has none,1
1,1,"the picture quality surprised me , when i firs...",0
2,2,great video clip quality for a digital camera ...,1
3,3,creative did well on its rechargeable battery ...,1
4,4,i highly recommend this camera to anyone looki...,1


In [ ]:
id = df_test['test_idx']
prediction = df_test['prediction']

file = "prediction_transf_1.json"

# Guardar en el json
nuwe_prediction_file(id,prediction,REL_PATH,"/data/processed/" ,file = file, colab = True)

#### Predicción con función

Se crea una función para crear predicciones del test con distintos modelos 

In [ ]:
transformer = f"finiteautomata/bertweet-base-sentiment-analysis"
MODEL = transformer
tokenizer = AutoTokenizer.from_pretrained(MODEL)

model = TFAutoModelForSequenceClassification.from_pretrained(MODEL)

df_test = ds_test.copy()

# Predicción con modelo y creación de json con la predicción en el formato nuwe
text_transformer_prediction(df_test, model, tokenizer, rel_path = REL_PATH, folder = "/data/processed/" ,file = "prediction_transf_2.json",colab=True)

All model checkpoint layers were used when initializing TFRobertaForSequenceClassification.

All the layers of TFRobertaForSequenceClassification were initialized from the model checkpoint at finiteautomata/bertweet-base-sentiment-analysis.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFRobertaForSequenceClassification for predictions without further training.


json saved


,test_idx,text,prediction
0,0,fm receiver it has none,1
1,1,"the picture quality surprised me , when i firs...",1
2,2,great video clip quality for a digital camera ...,1
3,3,creative did well on its rechargeable battery ...,1
4,4,i highly recommend this camera to anyone looki...,1
...,...,...,...
749,749,"the sound is awesome , but if u put it too lou...",1
750,750,the hitachi collet simply was not ground accur...,0
751,751,"b ) the lens cover is surely loose , i already...",0
752,752,"the keypad is a decent size , but the power on...",0


#### Predicción una vez entrenado con nuestros datos de train.csv

In [ ]:


# Configurar el modelo
model = TFRobertaForSequenceClassification.from_pretrained(f'roberta-base', num_labels=2)

optimizer = tf.keras.optimizers.Adam(learning_rate=3e-5)
loss = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)
metric = tf.keras.metrics.SparseCategoricalAccuracy('accuracy')

model.compile(optimizer=optimizer, loss=loss, metrics=[metric])

# Entrenar el modelo
history = model.fit(train_dataset.batch(32), epochs=2, validation_data=val_dataset.batch(32))


All model checkpoint layers were used when initializing TFRobertaForSequenceClassification.

Some layers of TFRobertaForSequenceClassification were not initialized from the model checkpoint at roberta-base and are newly initialized: ['classifier']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Epoch 1/2
76/76 [==============================] - 3753s 49s/step - loss: 0.4271 - accuracy: 0.8014 - val_loss: 0.2320 - val_accuracy: 0.9123
Epoch 2/2
76/76 [==============================] - 3212s 42s/step - loss: 0.2062 - accuracy: 0.9291 - val_loss: 0.2159 - val_accuracy: 0.9321


In [ ]:
history

In [ ]:
y_pred = model.predict(test_dataset.batch(16))


48/48 [==============================] - 307s 6s/step


In [ ]:
y_pred

TFSequenceClassifierOutput(loss=None, logits=array([[ 2.285104  , -2.2808843 ],
       [-0.54771394,  0.5441181 ],
       [-2.282097  ,  2.2231355 ],
       ...,
       [ 2.3647642 , -2.2882779 ],
       [ 0.9512195 , -1.1324998 ],
       [-2.0703082 ,  2.0324175 ]], dtype=float32), hidden_states=None, attentions=None)

In [ ]:
scores = y_pred
scores = softmax(scores)
res = np.argmax(scores)

In [ ]:
res

0

In [ ]:
y_predict = model.predict(test_dataset.batch(16))

48/48 [==============================] - 377s 8s/step


In [ ]:
y_predict

TFSequenceClassifierOutput(loss=None, logits=array([[ 2.285104  , -2.2808843 ],
       [-0.54771394,  0.5441181 ],
       [-2.282097  ,  2.2231355 ],
       ...,
       [ 2.3647642 , -2.2882779 ],
       [ 0.9512195 , -1.1324998 ],
       [-2.0703082 ,  2.0324175 ]], dtype=float32), hidden_states=None, attentions=None)

In [ ]:
t = softmax(y_predict[0][0])
t

array([0.98970747, 0.01029256], dtype=float32)

In [ ]:
r = np.argmax(t)
r

0

In [ ]:
len(y_predict[0])

754

In [ ]:
textos = y_predict[0]
y_pred = []

for i in range(len(textos)):
  t = textos[i]
  scores = softmax(t)
  res = np.argmax(scores)
    
  y_pred.append(res)

In [ ]:
test_df['prediction'] = y_pred

In [ ]:
id = ds_test['test_idx']
prediction = y_pred

file = "prediction_transf_3.json"

# Guardar en el json
nuwe_prediction_file(id, prediction, REL_PATH, "/data/processed/" ,file = file, colab = True)

json saved


## OBSERVACIONES:

- Se observa la eficiencia de los transformers incluso sin realizarle un entrenamiento con el dataset (train.csv) 

- Los resultados del f1-score en la web: 
  - F1-score modelo ROBERTA = 0.8154414205111185
  - F1-score modelo BERTWEET = 0.801
  - F1-score modelo ROBERTA entrenado = 0.91

- Mejora considerablemente respecto a los que probamos con los modelos supervisados del notebook anterior



